# Forecasting Process for ICOP

## First we read in packages for our workflow

In [1]:
import numpy as np

import pandas as pd

import statsmodels.api as sm

from statsmodels.tsa.api import VAR, DynamicVAR

from statsmodels.tsa.stattools import adfuller

import matplotlib.pyplot as plt

## Begin the Process for OC Forecasts
### Read in data - including some quick operations to clean up the excel

In [2]:
xl = pd.read_excel('.../3Q19DataViewOFC.xlsx',sheet_name="OC", usecols=range(1,9), index_col=0, skiprows=9, skipcol=1)
xl.drop(xl.tail(24).index,inplace=True) # drop forecast

In [3]:
xl.tail()

,Financial Activities (Jobs x 1000),Office-using Services (Jobs x 1000),Stock (SF x 1000),Completions (SF x 1000),Vacancy Rate (%),Net Absorption (SF x 1000),TW Rent Index ($/SF)
Year,,,,,,,
2018.3,112.7,299.9,75726,365,13.2,-106,32.81
2018.4,112.1,299.9,75726,0,12.6,460,32.75
2019.1,112.3,301.8,76066,340,12.7,242,32.89
2019.2,112.3,301.9,76628,562,12.4,700,33.26
2019.3,113.5,303.1,76976,348,12.3,442,33.52


## Here we create our calculated variables we will use in forecasting

In [4]:
xl['OFC Service Growth'] = xl['Office-using Services (Jobs x 1000)'].pct_change(periods=4) #stationary
xl['Finance Growth'] = xl['Financial Activities (Jobs x 1000)'].pct_change(periods=4) #stationary
xl['OFC Emp'] = xl['Financial Activities (Jobs x 1000)'] + xl['Office-using Services (Jobs x 1000)']
xl['OFC Emp Growth'] = xl['OFC Emp'].pct_change(periods=4) #stationary
xl['Comps %'] = xl['Completions (SF x 1000)']/xl['Stock (SF x 1000)']  #non-stationary
xl['Abs %'] = xl['Net Absorption (SF x 1000)']/xl['Stock (SF x 1000)'] #stationary
xl['Revenue'] = xl['TW Rent Index ($/SF)']*(1-(xl['Vacancy Rate (%)'])/100) #non-stationary
xl['LN Revenue'] = np.log(xl['Revenue']) 
xl['Revenue Growth'] = xl['Revenue'].pct_change(periods=4) #stationary
xl['LN Revenue Growth'] = xl['LN Revenue'].pct_change(periods=4) #stationary
xl['Comps MA %'] = xl['Comps %'].rolling(window=4).mean() #stationary
xl['Abs MA %'] = xl['Abs %'].rolling(window=4).mean() #stationary
# Vacancy Rate is non-stationary

In [5]:
xl.drop(xl.head(4).index,inplace=True) # drop n/a cells - they will break forecast

In [6]:
xl['Revenue Growth'].to_csv(".../OCHistorical.csv")

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [7]:
# xl['LN Revenue Growth'].plot(figsize=(15,4))

In [8]:
result = adfuller(xl['Revenue Growth'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -3.707870
p-value: 0.004004
Critical Values:
	1%: -3.484
	5%: -2.885
	10%: -2.579


### Here we prep the data that will go into the first model, select the model, fit the model, and export the model - it's a quick process

In [9]:
from statsmodels.tsa.base.datetools import dates_from_str

In [10]:
data = xl[['OFC Service Growth', 'Finance Growth', 'Comps MA %', 'Abs MA %', 'Revenue Growth']]

In [11]:
model = VAR(data)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [12]:
results = model.fit(maxlags=4, ic='bic')

In [13]:
fv = results.fittedvalues

In [14]:
fv.to_csv(".../OCBICFitted.csv")

In [15]:
## results.mean()

In [16]:
lag_order = results.k_ar
a = np.asarray(results.forecast(data.values[-lag_order:], 40))
np.savetxt(".../OCOFCforecastBIC.csv", a, delimiter=",")

### Repeat for different optimization

In [17]:
results = model.fit(maxlags=4, ic='aic')

In [18]:
fv = results.fittedvalues

In [19]:
fv.to_csv(".../OCAICFitted.csv")

In [20]:
lag_order = results.k_ar
a = np.asarray(results.forecast(data.values[-lag_order:], 40))
np.savetxt(".../OCOFCforecastAIC.csv", a, delimiter=",")

### Run next class of models

In [21]:
from statsmodels.tsa.arima_model import ARIMA

In [22]:
from statsmodels.tsa.arima_model import ARIMAResults

In [23]:
series = xl[['Revenue Growth']]

In [24]:
res = sm.tsa.arma_order_select_ic(series, ic=['aic', 'bic'], trend='nc')

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users

In [25]:
res.aic_min_order

(4, 1)

In [26]:
res.bic_min_order

(4, 1)

In [27]:
p = np.asarray(res.aic_min_order[0])

In [28]:
q = np.asarray(res.aic_min_order[1])

In [29]:
model = ARIMA(series, order = (p,0,q))
results_AR = model.fit(disp=0)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [30]:
fv = pd.DataFrame(results_AR.fittedvalues)

In [31]:
fv.to_csv(".../OCARMAAICFitted.csv")

In [32]:
df_forecast = pd.DataFrame(results_AR.forecast(40))

In [33]:
df_forecast.T.to_csv(".../OCOFCforecastARIMA1.csv")

### Repeat for different optimization

In [34]:
p = np.asarray(res.bic_min_order[0])

In [35]:
q = np.asarray(res.bic_min_order[1])

In [36]:
model = ARIMA(series, order = (p,0,q))
results_AR = model.fit(disp=0)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [37]:
fv = pd.DataFrame(results_AR.fittedvalues)

In [38]:
fv.to_csv(".../OCARMABICFitted.csv")

In [39]:
df_forecast = pd.DataFrame(results_AR.forecast(40))

In [40]:
df_forecast.T.to_csv(".../OCOFCforecastARIMA2.csv")

### Now we need to combine all the models into one file

In [41]:
files = {
    "date": pd.read_csv(".../Date.csv", squeeze=True),
    "VARAIC": pd.read_csv(".../OCOFCforecastAIC.csv", usecols=[4], header=None, names=["VARAIC"], squeeze=True),
    "ARIMA1": pd.read_csv(".../OCOFCforecastARIMA1.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA1"], squeeze=True),
    "ARIMA2": pd.read_csv(".../OCOFCforecastARIMA2.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA2"], squeeze=True),
    "VARBIC": pd.read_csv(".../OCOFCforecastBIC.csv", usecols=[4], header=None, names=["VARBIC"], squeeze=True)
}

In [42]:
df_OC = pd.concat([files["date"], files["VARAIC"], files["VARBIC"], files["ARIMA1"], files["ARIMA2"]], axis=1)

In [43]:
df_OC = df_OC.set_index("Date")

### Here we create a simple average of all of the models. Depending on optimization - this gives us 2-5 distinct models to choose from.

In [44]:
df_OC['AVERAGE'] = df_OC.mean(axis=1)
df_OC.head()

,VARAIC,VARBIC,ARIMA1,ARIMA2,AVERAGE
Date,,,,,
Q4 2019,0.034908,0.030715,0.035920,0.035920,0.034366
Q1 2020,0.036306,0.028992,0.036285,0.036285,0.034467
Q2 2020,0.032248,0.027292,0.036807,0.036807,0.033289
Q3 2020,0.026079,0.025766,0.034972,0.034972,0.030448
Q4 2020,0.026606,0.024502,0.033474,0.033474,0.029514


## Repeat this process for the SV forecasts
### Read in data - including some quick operations to clean up the excel

In [45]:
xl = pd.read_excel('.../3Q19DataViewOFC.xlsx',sheet_name="SV", usecols=range(1,9), index_col=0, skiprows=9, skipcol=1)
xl.drop(xl.tail(24).index,inplace=True) # drop forecast

## Here we create our calculated variables we will use in forecasting

In [46]:
xl['OFC Service Growth'] = xl['Office-using Services (Jobs x 1000)'].pct_change(periods=4) #stationary
xl['Finance Growth'] = xl['Financial Activities (Jobs x 1000)'].pct_change(periods=4) #stationary
xl['OFC Emp'] = xl['Financial Activities (Jobs x 1000)'] + xl['Office-using Services (Jobs x 1000)']
xl['OFC Emp Growth'] = xl['OFC Emp'].pct_change(periods=4) #stationary
xl['Comps %'] = xl['Completions (SF x 1000)']/xl['Stock (SF x 1000)']  #stationary
xl['Abs %'] = xl['Net Absorption (SF x 1000)']/xl['Stock (SF x 1000)'] #stationary
xl['Revenue'] = xl['TW Rent Index ($/SF)']*(1-(xl['Vacancy Rate (%)'])/100) 
xl['LN Revenue'] = np.log(xl['Revenue']) 
xl['Revenue Growth'] = xl['Revenue'].pct_change(periods=4) #stationary
xl['LN Revenue Growth'] = xl['LN Revenue'].pct_change(periods=4) #stationary
xl['Comps MA %'] = xl['Comps %'].rolling(window=4).mean() #stationary
xl['Abs MA %'] = xl['Abs %'].rolling(window=4).mean() #non-stationary
# Vacancy Rate is stationary

In [47]:
xl.drop(xl.head(4).index,inplace=True) # drop n/a cells - they will break forecast

In [48]:
xl['Revenue Growth'].to_csv(".../SVHistorical.csv")

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [49]:
result = adfuller(xl['Vacancy Rate (%)'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -3.792756
p-value: 0.002982
Critical Values:
	1%: -3.481
	5%: -2.884
	10%: -2.579


### Here we prep the data that will go into the first model, select the model, fit the model, and export the model - it's a quick process

In [50]:
from statsmodels.tsa.base.datetools import dates_from_str

In [51]:
data = xl[['OFC Service Growth', 'Finance Growth', 'Comps MA %', 'Abs MA %', 'Revenue Growth']]

In [52]:
model = VAR(data)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [53]:
results = model.fit(maxlags=4, ic='bic')

In [54]:
fv = results.fittedvalues

In [55]:
fv.to_csv(".../SVBICFitted.csv")

In [56]:
lag_order = results.k_ar
a = np.asarray(results.forecast(data.values[-lag_order:], 40))
np.savetxt(".../SVOFCforecastBIC.csv", a, delimiter=",")

### Repeat for different optimization

In [57]:
results = model.fit(maxlags=4, ic='aic')

In [58]:
fv = results.fittedvalues

In [59]:
fv.to_csv(".../SVAICFitted.csv")

In [60]:
lag_order = results.k_ar
a = np.asarray(results.forecast(data.values[-lag_order:], 40))
np.savetxt(".../SVOFCforecastAIC.csv", a, delimiter=",")

### Run next class of models

In [61]:
from statsmodels.tsa.arima_model import ARIMA

In [62]:
from statsmodels.tsa.arima_model import ARIMAResults

In [63]:
series = xl[['Revenue Growth']]

In [64]:
res = sm.tsa.arma_order_select_ic(series, ic=['aic', 'bic'], trend='nc')

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users

In [65]:
res.aic_min_order

(4, 2)

In [66]:
res.bic_min_order

(2, 2)

In [67]:
p = np.asarray(res.aic_min_order[0])

In [68]:
q = np.asarray(res.aic_min_order[1])

In [69]:
model = ARIMA(series, order = (p,0,q))
results_AR = model.fit(disp=0)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [70]:
fv = pd.DataFrame(results_AR.fittedvalues)

In [71]:
fv.to_csv(".../SVARMAAICFitted.csv")

In [72]:
df_forecast = pd.DataFrame(results_AR.forecast(40))

In [73]:
df_forecast.T.to_csv(".../SVOFCforecastARIMA1.csv")

### Repeat for different optimization

In [74]:
p = np.asarray(res.bic_min_order[0])

In [75]:
q = np.asarray(res.bic_min_order[1])

In [76]:
model = ARIMA(series, order = (p,0,q))
results_AR = model.fit(disp=0)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [77]:
fv = pd.DataFrame(results_AR.fittedvalues)

In [78]:
fv.to_csv(".../SVARMABICFitted.csv")

In [79]:
df_forecast = pd.DataFrame(results_AR.forecast(40))

In [80]:
df_forecast.T.to_csv(".../SVOFCforecastARIMA2.csv")

### Now we need to combine all the models into one file

In [81]:
files = {
    "date": pd.read_csv(".../Date.csv", squeeze=True),
    "VARAIC": pd.read_csv(".../SVOFCforecastAIC.csv", usecols=[4], header=None, names=["VARAIC"], squeeze=True),
    "ARIMA1": pd.read_csv(".../SVOFCforecastARIMA1.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA1"], squeeze=True),
    "ARIMA2": pd.read_csv(".../SVOFCforecastARIMA2.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA2"], squeeze=True),
    "VARBIC": pd.read_csv(".../SVOFCforecastBIC.csv", usecols=[4], header=None, names=["VARBIC"], squeeze=True)
}

In [82]:
df_SV = pd.concat([files["date"], files["VARAIC"], files["VARBIC"], files["ARIMA1"], files["ARIMA2"]], axis=1)

In [83]:
df_SV = df_SV.set_index("Date")

### Here we create a simple average of all of the models. Depending on optimization - this gives us 2-5 distinct models to choose from.

In [84]:
df_SV['AVERAGE'] = df_SV.mean(axis=1)
df_SV.head()

,VARAIC,VARBIC,ARIMA1,ARIMA2,AVERAGE
Date,,,,,
Q4 2019,0.072556,0.072556,0.090852,0.090642,0.081651
Q1 2020,0.050484,0.050484,0.064047,0.066919,0.057984
Q2 2020,0.038337,0.038337,0.039925,0.046290,0.040723
Q3 2020,0.032630,0.032630,0.028297,0.032784,0.031585
Q4 2020,0.030000,0.030000,0.022395,0.026639,0.027258


## Repeat this process for the SD forecasts
### Read in data - including some quick operations to clean up the excel

In [85]:
xl = pd.read_excel('.../3Q19DataViewOFC.xlsx',sheet_name="SD", usecols=range(1,9), index_col=0, skiprows=9, skipcol=1)
xl.drop(xl.tail(24).index,inplace=True) # drop forecast

## Here we create our calculated variables we will use in forecasting

In [86]:
xl['OFC Service Growth'] = xl['Office-using Services (Jobs x 1000)'].pct_change(periods=4) #non-stationary
xl['Finance Growth'] = xl['Financial Activities (Jobs x 1000)'].pct_change(periods=4) #non-stationary
xl['OFC Emp'] = xl['Financial Activities (Jobs x 1000)'] + xl['Office-using Services (Jobs x 1000)']
xl['OFC Emp Growth'] = xl['OFC Emp'].pct_change(periods=4) #non-stationary
xl['Comps %'] = xl['Completions (SF x 1000)']/xl['Stock (SF x 1000)']  #stationary
xl['Abs %'] = xl['Net Absorption (SF x 1000)']/xl['Stock (SF x 1000)'] #stationary
xl['Revenue'] = xl['TW Rent Index ($/SF)']*(1-(xl['Vacancy Rate (%)'])/100) 
xl['LN Revenue'] = np.log(xl['Revenue']) 
xl['Revenue Growth'] = xl['Revenue'].pct_change(periods=4) #non-stationary
xl['LN Revenue Growth'] = xl['LN Revenue'].pct_change(periods=4) #non-stationary
xl['Comps MA %'] = xl['Comps %'].rolling(window=4).mean() #non-stationary
xl['Abs MA %'] = xl['Abs %'].rolling(window=4).mean() #non-stationary
# Vacancy Rate is non-stationary

In [87]:
xl.drop(xl.head(4).index,inplace=True) # drop n/a cells - they will break forecast

In [88]:
xl['Revenue Growth'].to_csv(".../SDHistorical.csv")

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [89]:
result = adfuller(xl['Abs MA %'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -2.502258
p-value: 0.114926
Critical Values:
	1%: -3.482
	5%: -2.884
	10%: -2.579


### Here we prep the data that will go into the first model, select the model, fit the model, and export the model - it's a quick process

In [90]:
from statsmodels.tsa.base.datetools import dates_from_str

In [91]:
data = xl[['OFC Service Growth', 'Finance Growth', 'Comps MA %', 'Abs MA %', 'Revenue Growth']]

In [92]:
model = VAR(data)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [93]:
results = model.fit(maxlags=4, ic='bic')

In [94]:
fv = results.fittedvalues

In [95]:
fv.to_csv(".../SDBICFitted.csv")

In [96]:
lag_order = results.k_ar
a = np.asarray(results.forecast(data.values[-lag_order:], 40))
np.savetxt(".../SDOFCforecastBIC.csv", a, delimiter=",")

### Repeat for different optimization

In [97]:
results = model.fit(maxlags=4, ic='aic')

In [98]:
fv = results.fittedvalues

In [99]:
fv.to_csv(".../SDAICFitted.csv")

In [100]:
lag_order = results.k_ar
a = np.asarray(results.forecast(data.values[-lag_order:], 40))
np.savetxt(".../SDOFCforecastAIC.csv", a, delimiter=",")

### Run next class of models

In [101]:
from statsmodels.tsa.arima_model import ARIMA

In [102]:
from statsmodels.tsa.arima_model import ARIMAResults

In [103]:
series = xl[['Revenue Growth']]

In [104]:
res = sm.tsa.arma_order_select_ic(series, ic=['aic', 'bic'], trend='nc')

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users

In [105]:
res.aic_min_order

(4, 2)

In [106]:
res.bic_min_order

(4, 2)

In [107]:
p = np.asarray(res.aic_min_order[0])

In [108]:
q = np.asarray(res.aic_min_order[1])

In [109]:
model = ARIMA(series, order = (p,0,q))
results_AR = model.fit(disp=0)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [110]:
fv = pd.DataFrame(results_AR.fittedvalues)

In [111]:
fv.to_csv(".../SDARMAAICFitted.csv")

In [112]:
df_forecast = pd.DataFrame(results_AR.forecast(40))

In [113]:
df_forecast.T.to_csv(".../SDOFCforecastARIMA1.csv")

### Repeat for different optimization

In [114]:
p = np.asarray(res.bic_min_order[0])

In [115]:
q = np.asarray(res.bic_min_order[1])

In [116]:
model = ARIMA(series, order = (p,0,q))
results_AR = model.fit(disp=0)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [117]:
fv = pd.DataFrame(results_AR.fittedvalues)

In [118]:
fv.to_csv(".../SDARMABICFitted.csv")

In [119]:
df_forecast = pd.DataFrame(results_AR.forecast(40))

In [120]:
df_forecast.T.to_csv(".../SDOFCforecastARIMA2.csv")

### Now we need to combine all the models into one file

In [121]:
files = {
    "date": pd.read_csv(".../Date.csv", squeeze=True),
    "VARAIC": pd.read_csv(".../SDOFCforecastAIC.csv", usecols=[4], header=None, names=["VARAIC"], squeeze=True),
    "ARIMA1": pd.read_csv(".../SDOFCforecastARIMA1.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA1"], squeeze=True),
    "ARIMA2": pd.read_csv(".../SDOFCforecastARIMA2.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA2"], squeeze=True),
    "VARBIC": pd.read_csv(".../SDOFCforecastBIC.csv", usecols=[4], header=None, names=["VARBIC"], squeeze=True)
}

In [122]:
df_SD = pd.concat([files["date"], files["VARAIC"], files["VARBIC"], files["ARIMA1"], files["ARIMA2"]], axis=1)

In [123]:
df_SD = df_SD.set_index("Date")

### Here we create a simple average of all of the models. Depending on optimization - this gives us 2-5 distinct models to choose from.

In [124]:
df_SD['AVERAGE'] = df_SD.mean(axis=1)
df_SD.head()

,VARAIC,VARBIC,ARIMA1,ARIMA2,AVERAGE
Date,,,,,
Q4 2019,0.027265,0.025838,0.029583,0.029583,0.028067
Q1 2020,0.033641,0.026490,0.027513,0.027513,0.028789
Q2 2020,0.039512,0.027401,0.022445,0.022445,0.027951
Q3 2020,0.042715,0.028384,0.021939,0.021939,0.028744
Q4 2020,0.043355,0.029319,0.023298,0.023298,0.029817


## Repeat this process for the CHI forecasts
### Read in data - including some quick operations to clean up the excel

In [125]:
xl = pd.read_excel('.../3Q19DataViewOFC.xlsx',sheet_name="CHI", usecols=range(1,9), index_col=0, skiprows=9, skipcol=1)
xl.drop(xl.tail(24).index,inplace=True) # drop forecast

## Here we create our calculated variables we will use in forecasting

In [126]:
xl['OFC Service Growth'] = xl['Office-using Services (Jobs x 1000)'].pct_change(periods=4) #non-stationary
xl['Finance Growth'] = xl['Financial Activities (Jobs x 1000)'].pct_change(periods=4) #non-stationary
xl['OFC Emp'] = xl['Financial Activities (Jobs x 1000)'] + xl['Office-using Services (Jobs x 1000)']
xl['OFC Emp Growth'] = xl['OFC Emp'].pct_change(periods=4) #non-stationary
xl['Comps %'] = xl['Completions (SF x 1000)']/xl['Stock (SF x 1000)']  #stationary
xl['Abs %'] = xl['Net Absorption (SF x 1000)']/xl['Stock (SF x 1000)'] #stationary
xl['Revenue'] = xl['TW Rent Index ($/SF)']*(1-(xl['Vacancy Rate (%)'])/100) 
xl['LN Revenue'] = np.log(xl['Revenue']) 
xl['Revenue Growth'] = xl['Revenue'].pct_change(periods=4) #non-stationary
xl['LN Revenue Growth'] = xl['LN Revenue'].pct_change(periods=4) #non-stationary
xl['Comps MA %'] = xl['Comps %'].rolling(window=4).mean() #non-stationary
xl['Abs MA %'] = xl['Abs %'].rolling(window=4).mean() #non-stationary
# Vacancy Rate is non-stationary

In [127]:
xl.drop(xl.head(4).index,inplace=True) # drop n/a cells - they will break forecast

In [128]:
xl['Revenue Growth'].to_csv(".../CHIHistorical.csv")

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [129]:
result = adfuller(xl['Abs MA %'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -3.464083
p-value: 0.008959
Critical Values:
	1%: -3.492
	5%: -2.888
	10%: -2.581


### Here we prep the data that will go into the first model, select the model, fit the model, and export the model - it's a quick process

In [130]:
from statsmodels.tsa.base.datetools import dates_from_str

In [131]:
data = xl[['OFC Service Growth', 'Finance Growth', 'Comps MA %', 'Abs MA %', 'Revenue Growth']]

In [132]:
model = VAR(data)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [133]:
results = model.fit(maxlags=4, ic='bic')

In [134]:
fv = results.fittedvalues

In [135]:
fv.to_csv(".../CHIBICFitted.csv")

In [136]:
lag_order = results.k_ar
a = np.asarray(results.forecast(data.values[-lag_order:], 40))
np.savetxt(".../CHIOFCforecastBIC.csv", a, delimiter=",")

### Repeat for different optimization

In [137]:
results = model.fit(maxlags=4, ic='aic')

In [138]:
fv = results.fittedvalues

In [139]:
fv.to_csv(".../CHIAICFitted.csv")

In [140]:
lag_order = results.k_ar
a = np.asarray(results.forecast(data.values[-lag_order:], 40))
np.savetxt(".../CHIOFCforecastAIC.csv", a, delimiter=",")

### Run next class of models

In [141]:
from statsmodels.tsa.arima_model import ARIMA

In [142]:
from statsmodels.tsa.arima_model import ARIMAResults

In [143]:
series = xl[['Revenue Growth']]

In [144]:
res = sm.tsa.arma_order_select_ic(series, ic=['aic', 'bic'], trend='nc')

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users

In [145]:
res.aic_min_order

(4, 2)

In [146]:
res.bic_min_order

(3, 2)

In [147]:
p = np.asarray(res.aic_min_order[0])

In [148]:
q = np.asarray(res.aic_min_order[1])

In [149]:
model = ARIMA(series, order = (p,0,q))
results_AR = model.fit(disp=0)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [150]:
fv = pd.DataFrame(results_AR.fittedvalues)

In [151]:
fv.to_csv(".../CHIARMAAICFitted.csv")

In [152]:
df_forecast = pd.DataFrame(results_AR.forecast(40))

In [153]:
df_forecast.T.to_csv(".../CHIOFCforecastARIMA1.csv")

### Repeat for different optimization

In [154]:
p = np.asarray(res.bic_min_order[0])

In [155]:
q = np.asarray(res.bic_min_order[1])

In [156]:
model = ARIMA(series, order = (p,0,q))
results_AR = model.fit(disp=0)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [157]:
fv = pd.DataFrame(results_AR.fittedvalues)

In [158]:
fv.to_csv(".../CHIARMABICFitted.csv")

In [159]:
df_forecast = pd.DataFrame(results_AR.forecast(40))

In [160]:
df_forecast.T.to_csv(".../CHIOFCforecastARIMA2.csv")

### Now we need to combine all the models into one file

In [161]:
files = {
    "date": pd.read_csv(".../Date.csv", squeeze=True),
    "VARAIC": pd.read_csv(".../CHIOFCforecastAIC.csv", usecols=[4], header=None, names=["VARAIC"], squeeze=True),
    "ARIMA1": pd.read_csv(".../CHIOFCforecastARIMA1.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA1"], squeeze=True),
    "ARIMA2": pd.read_csv(".../CHIOFCforecastARIMA2.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA2"], squeeze=True),
    "VARBIC": pd.read_csv(".../CHIOFCforecastBIC.csv", usecols=[4], header=None, names=["VARBIC"], squeeze=True)
}

In [162]:
df_CHI = pd.concat([files["date"], files["VARAIC"], files["VARBIC"], files["ARIMA1"], files["ARIMA2"]], axis=1)

In [163]:
df_CHI = df_CHI.set_index("Date")

### Here we create a simple average of all of the models. Depending on optimization - this gives us 2-5 distinct models to choose from.

In [164]:
df_CHI['AVERAGE'] = df_CHI.mean(axis=1)
df_CHI.head()

,VARAIC,VARBIC,ARIMA1,ARIMA2,AVERAGE
Date,,,,,
Q4 2019,0.027469,0.029812,0.035089,0.031795,0.031041
Q1 2020,0.023876,0.025883,0.034190,0.028708,0.028164
Q2 2020,0.021868,0.022619,0.028931,0.027230,0.025162
Q3 2020,0.020196,0.019892,0.025649,0.026601,0.023084
Q4 2020,0.018680,0.017614,0.025040,0.025674,0.021752


## Repeat this process for the WLA forecasts
### Read in data - including some quick operations to clean up the excel

In [165]:
xl = pd.read_excel('.../3Q19DataViewOFC.xlsx',sheet_name="WLA", usecols=range(1,9), index_col=0, skiprows=9, skipcol=1)
xl.drop(xl.tail(24).index,inplace=True) # drop forecast

## Here we create our calculated variables we will use in forecasting

In [166]:
xl['OFC Service Growth'] = xl['Office-using Services (Jobs x 1000)'].pct_change(periods=4) #non-stationary
xl['Finance Growth'] = xl['Financial Activities (Jobs x 1000)'].pct_change(periods=4) #non-stationary
xl['OFC Emp'] = xl['Financial Activities (Jobs x 1000)'] + xl['Office-using Services (Jobs x 1000)']
xl['OFC Emp Growth'] = xl['OFC Emp'].pct_change(periods=4) #non-stationary
xl['Comps %'] = xl['Completions (SF x 1000)']/xl['Stock (SF x 1000)']  #stationary
xl['Abs %'] = xl['Net Absorption (SF x 1000)']/xl['Stock (SF x 1000)'] #stationary
xl['Revenue'] = xl['TW Rent Index ($/SF)']*(1-(xl['Vacancy Rate (%)'])/100) 
xl['LN Revenue'] = np.log(xl['Revenue']) 
xl['Revenue Growth'] = xl['Revenue'].pct_change(periods=4) #non-stationary
xl['LN Revenue Growth'] = xl['LN Revenue'].pct_change(periods=4) #non-stationary
xl['Comps MA %'] = xl['Comps %'].rolling(window=4).mean() #non-stationary
xl['Abs MA %'] = xl['Abs %'].rolling(window=4).mean() #non-stationary
# Vacancy Rate is non-stationary

In [167]:
xl.drop(xl.head(4).index,inplace=True) # drop n/a cells - they will break forecast

In [168]:
xl['Revenue Growth'].to_csv(".../WLAHistorical.csv")

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [169]:
result = adfuller(xl['Abs MA %'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -3.074124
p-value: 0.028528
Critical Values:
	1%: -3.489
	5%: -2.887
	10%: -2.580


### Here we prep the data that will go into the first model, select the model, fit the model, and export the model - it's a quick process

In [170]:
from statsmodels.tsa.base.datetools import dates_from_str

In [171]:
data = xl[['OFC Service Growth', 'Finance Growth', 'Comps MA %', 'Abs MA %', 'Revenue Growth']]

In [172]:
model = VAR(data)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [173]:
results = model.fit(maxlags=4, ic='bic')

In [174]:
fv = results.fittedvalues

In [175]:
fv.to_csv(".../WLABICFitted.csv")

In [176]:
lag_order = results.k_ar
a = np.asarray(results.forecast(data.values[-lag_order:], 40))
np.savetxt(".../WLAOFCforecastBIC.csv", a, delimiter=",")

### Repeat for different optimization

In [177]:
results = model.fit(maxlags=4, ic='aic')

In [178]:
fv = results.fittedvalues

In [179]:
fv.to_csv(".../WLAAICFitted.csv")

In [180]:
lag_order = results.k_ar
a = np.asarray(results.forecast(data.values[-lag_order:], 40))
np.savetxt(".../WLAOFCforecastAIC.csv", a, delimiter=",")

### Run next class of models

In [181]:
from statsmodels.tsa.arima_model import ARIMA

In [182]:
from statsmodels.tsa.arima_model import ARIMAResults

In [183]:
series = xl[['Revenue Growth']]

In [184]:
res = sm.tsa.arma_order_select_ic(series, ic=['aic', 'bic'], trend='nc')

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users

In [185]:
res.aic_min_order

(4, 1)

In [186]:
res.bic_min_order

(4, 1)

In [187]:
p = np.asarray(res.aic_min_order[0])

In [188]:
q = np.asarray(res.aic_min_order[1])

In [189]:
model = ARIMA(series, order = (p,0,q))
results_AR = model.fit(disp=0)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [190]:
fv = pd.DataFrame(results_AR.fittedvalues)

In [191]:
fv.to_csv(".../WLAARMAAICFitted.csv")

In [192]:
df_forecast = pd.DataFrame(results_AR.forecast(40))

In [193]:
df_forecast.T.to_csv(".../WLAOFCforecastARIMA1.csv")

### Repeat for different optimization

In [194]:
p = np.asarray(res.bic_min_order[0])

In [195]:
q = np.asarray(res.bic_min_order[1])

In [196]:
model = ARIMA(series, order = (p,0,q))
results_AR = model.fit(disp=0)

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [197]:
fv = pd.DataFrame(results_AR.fittedvalues)

In [198]:
fv.to_csv(".../WLAARMABICFitted.csv")

In [199]:
df_forecast = pd.DataFrame(results_AR.forecast(40))

In [200]:
df_forecast.T.to_csv(".../WLAOFCforecastARIMA2.csv")

### Now we need to combine all the models into one file

In [201]:
files = {
    "date": pd.read_csv(".../Date.csv", squeeze=True),
    "VARAIC": pd.read_csv(".../WLAOFCforecastAIC.csv", usecols=[4], header=None, names=["VARAIC"], squeeze=True),
    "ARIMA1": pd.read_csv(".../WLAOFCforecastARIMA1.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA1"], squeeze=True),
    "ARIMA2": pd.read_csv(".../WLAOFCforecastARIMA2.csv", usecols=[1], skiprows=[0], header=None, names=["ARIMA2"], squeeze=True),
    "VARBIC": pd.read_csv(".../WLAOFCforecastBIC.csv", usecols=[4], header=None, names=["VARBIC"], squeeze=True)
}

In [202]:
df_WLA = pd.concat([files["date"], files["VARAIC"], files["VARBIC"], files["ARIMA1"], files["ARIMA2"]], axis=1)

In [203]:
df_WLA = df_WLA.set_index("Date")

### Here we create a simple average of all of the models. Depending on optimization - this gives us 2-5 distinct models to choose from.

In [204]:
df_WLA['AVERAGE'] = df_WLA.mean(axis=1)
df_WLA.head()

,VARAIC,VARBIC,ARIMA1,ARIMA2,AVERAGE
Date,,,,,
Q4 2019,0.066425,0.071917,0.048041,0.048041,0.058606
Q1 2020,0.080711,0.082657,0.034123,0.034123,0.057903
Q2 2020,0.085655,0.087747,0.025772,0.025772,0.056236
Q3 2020,0.085731,0.088363,0.021189,0.021189,0.054118
Q4 2020,0.081411,0.085569,0.014966,0.014966,0.049228


### Export into working folder

In [205]:
writer = pd.ExcelWriter('.../RevPAF Forecasts.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_OC.to_excel(writer, sheet_name='OC Forecast')
df_SV.to_excel(writer, sheet_name='SV Forecast')
df_SD.to_excel(writer, sheet_name='SD Forecast')
df_CHI.to_excel(writer, sheet_name='CHI Forecast')
df_WLA.to_excel(writer, sheet_name='WLA Forecast')

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheetOC = writer.sheets['OC Forecast']
worksheetSV = writer.sheets['SV Forecast']
worksheetSD = writer.sheets['SD Forecast']
worksheetCHI = writer.sheets['CHI Forecast']
worksheetWLA = writer.sheets['WLA Forecast']
format1 = workbook.add_format({'num_format': '0.0%'})

# Set the format but not the column width.
worksheetOC.set_column('B:F', None, format1)
worksheetSV.set_column('B:F', None, format1)
worksheetSD.set_column('B:F', None, format1)
worksheetCHI.set_column('B:F', None, format1)
worksheetWLA.set_column('B:F', None, format1)

# Create a chart object.
OC_line_chart = workbook.add_chart({'type': 'line'})
SV_line_chart = workbook.add_chart({'type': 'line'})
SD_line_chart = workbook.add_chart({'type': 'line'})
CHI_line_chart = workbook.add_chart({'type': 'line'})
WLA_line_chart = workbook.add_chart({'type': 'line'})

# Configure the series of the chart from the dataframe data.
OC_line_chart.add_series({
    'name':       'OC Forecast Avg',
    'categories': '=OC Forecast!$A$2:$A$41',
    'values':     '=OC Forecast!$F$2:$F$41',
})

SV_line_chart.add_series({
    'name':       'SV Forecast Avg',
    'categories': '=SV Forecast!$A$2:$A$41',
    'values':     '=SV Forecast!$F$2:$F$41',
})

SD_line_chart.add_series({
    'name':       'SD Forecast Avg',
    'categories': '=SD Forecast!$A$2:$A$41',
    'values':     '=SD Forecast!$F$2:$F$41',
})

CHI_line_chart.add_series({
    'name':       'CHI Forecast Avg',
    'categories': '=CHI Forecast!$A$2:$A$41',
    'values':     '=CHI Forecast!$F$2:$F$41',
})

WLA_line_chart.add_series({
    'name':       'WLA Forecast Avg',
    'categories': '=WLA Forecast!$A$2:$A$41',
    'values':     '=WLA Forecast!$F$2:$F$41',
})

# # Combine the charts.
# OC_line_chart.combine(SV_line_chart)

# Insert the chart into the worksheet.
worksheetOC.insert_chart('H2', OC_line_chart)
worksheetSV.insert_chart('H2', SV_line_chart)
worksheetSD.insert_chart('H2', SD_line_chart)
worksheetCHI.insert_chart('H2', CHI_line_chart)
worksheetWLA.insert_chart('H2', WLA_line_chart)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\xlsxwriter\chart.py:836: UserWarning: Sheetname in '=OC Forecast!$F$2:$F$41' contains spaces but isn't quoted. This may cause errors in Excel.
  "This may cause errors in Excel." % data)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\xlsxwriter\chart.py:836: UserWarning: Sheetname in '=OC Forecast!$A$2:$A$41' contains spaces but isn't quoted. This may cause errors in Excel.
  "This may cause errors in Excel." % data)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\xlsxwriter\chart.py:836: UserWarning: Sheetname in '=SV Forecast!$F$2:$F$41' contains spaces but isn't quoted. This may cause errors in Excel.
  "This may cause errors in Excel." % data)
C:\Users\msaia\AppData\Local\Continuum\anaconda3\lib\site-packages\xlsxwriter\chart.py:836: UserWarning: Sheetname in '=SV Forecast!$A$2:$A$41' contains spaces but isn't quoted. This may cause errors in Excel.
  "This may cause errors in 